In [ ]:
# Code derived from: https://github.com/bioinf-jku/TTUR
from __future__ import absolute_import, division, print_function
import os
import glob
import numpy as np
import fid

import tensorflow.compat.v1 as tf # The is score was made in tensorflow v1
tf.disable_v2_behavior()          # These two lines solves the problem

# Paths
samples_path = '.generated_samples/msggan/afhq/'
stats_path = './fid_stats-afhq64x64.npz' # training set statistics
inception_path = fid.check_or_download_inception(None) # download inception network

In [ ]:
out_path = samples_path
res = []
for file in os.listdir(out_path):
    print(out_path + file)
    res.append(np.load(out_path + file))
        
numpy_data = np.empty((0, ) + res[0].shape[1:])
for img in res:
    numpy_data = np.concatenate((numpy_data, img))

input_list = []
for image in numpy_data:
    input_list.append(image)

In [ ]:
images = np.array(input_list)

In [ ]:
# load precalculated training set statistics
f = np.load(stats_path)
mu_real, sigma_real = f['mu'][:], f['sigma'][:]
f.close()

fid.create_inception_graph(inception_path)  # load the graph into the current TF graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    mu_gen, sigma_gen = fid.calculate_activation_statistics(images, sess, batch_size=100)

fid_value = fid.calculate_frechet_distance(mu_gen, sigma_gen, mu_real, sigma_real)
print("FID: %s" % fid_value)